In [22]:
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from gcn_utils import collate
from utils import get_data
from tqdm import tqdm



In [50]:
import math
import torch
import torch.nn as nn
import dgl
import dgl.function as fn


def msg_func(edges):
    return {'m': edges.data['weight'].unsqueeze(1) * edges.src['h']}


class GCNLayer(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 activation,
                 dropout,
                 bias=True):
        super(GCNLayer, self).__init__()

        self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.bias = None
        self.activation = activation
        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = False

        self.norm = 'none'

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, h, g):
        if self.dropout:
            h = self.dropout(h)

        g.srcdata['h'] = h
        g.update_all(msg_func, fn.sum(msg='m', out='h'))
        h = g.dstdata['h']
        # bias

        h = torch.mm(h, self.weight)
        if self.norm != 'none':
            degs = g.in_degrees().to(h.device).float().clamp(min=1)
            if self.norm == 'both':
                norm = torch.pow(degs, -0.5)
            else:
                norm = 1.0 / degs
            shp = norm.shape + (1,) * (h.dim() - 1)
            norm = torch.reshape(norm, shp)
            h = h * norm

        if self.bias is not None:
            h = h + self.bias
        if self.activation:
            h = self.activation(h)
        return h


class GCN(nn.Module):
    def __init__(self,
                 num_element,
                 embedding_dim,
                 hidden_dim,
                 num_classes,
                 num_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.element_emebdding = nn.Embedding(num_element, embedding_dim)
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(GCNLayer(embedding_dim, hidden_dim, activation, False))
        # hidden layers
        for i in range(num_layers - 1):
            self.layers.append(GCNLayer(hidden_dim, hidden_dim, activation, dropout))
        # output layer
        self.out = nn.Linear(hidden_dim, num_classes)

    def forward(self, g):
        h = self.element_emebdding(g.ndata['elem_idx'])
        for i, layer in enumerate(self.layers):
            h = layer(h, g)
            g.ndata[f'layer_0'] = h
        g.ndata['h'] = h
        # Calculate graph representation by averaging all the node representations.
        hg = dgl.mean_nodes(g, 'h')
        return self.out(hg)


In [2]:
train_data = get_data('gdb_9_clean.tsv', dataset='gdb-9', device='cuda:0')
data_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate)
model = GCN(27, 16, 32, 11, 2, F.relu, 0.1)
loss_func = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
device = torch.device('cuda:0')
model.to(device)

Reading raw data ... : : 133885it [01:21, 1637.87it/s]
Converting to DGL graphs ...: 100%|███████████████████████████████████████████| 133885/133885 [03:15<00:00, 685.07it/s]


GCN(
  (element_emebdding): Embedding(27, 16)
  (layers): ModuleList(
    (0): GCNLayer()
    (1): GCNLayer(
      (dropout): Dropout(p=0.1)
    )
  )
  (out): Linear(in_features=32, out_features=11, bias=True)
)

In [53]:
model = GCN(27, 16, 32, 11, 2, F.relu, 0.1)
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
device = torch.device('cuda:0')
model.to(device)

GCN(
  (element_emebdding): Embedding(27, 16)
  (layers): ModuleList(
    (0): GCNLayer()
    (1): GCNLayer(
      (dropout): Dropout(p=0.1)
    )
  )
  (out): Linear(in_features=32, out_features=11, bias=True)
)

In [54]:
for epoch in range(80):
    epoch_loss = 0
    batch = tqdm(data_loader)
    for bg, label in batch:

        prediction = model(bg)
        loss = loss_func(prediction, label) # * (1 / torch.pow(label, 2)))
        # print(bg.ndata['layer_0'])
        # print(bg.ndata[f'layer_0'])
        # print(1 / torch.pow(label, 2))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()
        batch.set_description(f'epoch {epoch} loss {epoch_loss}')
    
    
    if epoch > 10:
        print(prediction.detach().cpu())
        print(label.cpu())
































  0%|                                                                                         | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 199168.0625:   0%|                                                               | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 398529.703125:   0%|                                                             | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 592674.890625:   0%|                                                             | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 784684.65625:   0%|                                                              | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 964285.953125:   0%|                                                             | 0/4184 [00:00<?, ?it/s]





























epoch 0 loss 1156631.3125:   0%|                                            

epoch 0 loss 9342196.765625:   1%|▌                                                  | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 9574108.375:   1%|▌                                                     | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 9766629.6875:   1%|▌                                                    | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 9955825.296875:   1%|▌                                                  | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 10154600.953125:   1%|▌                                                 | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 10339919.4375:   1%|▌                                                   | 46/4184 [00:00<00:59, 69.16it/s]





























epoch 0 loss 10537120.59375:   1%|▌                                                  | 46/4184 [00:00<00:5

epoch 0 loss 18023256.9609375:   2%|█                                                | 93/4184 [00:01<00:57, 70.81it/s]





























epoch 0 loss 18154963.1015625:   2%|█                                                | 93/4184 [00:01<00:57, 70.81it/s]





























epoch 0 loss 18289158.0546875:   2%|█                                                | 93/4184 [00:01<00:57, 70.81it/s]





























epoch 0 loss 18418024.3125:   2%|█▏                                                  | 93/4184 [00:01<00:57, 70.81it/s]





























epoch 0 loss 18542681.2421875:   2%|█                                                | 93/4184 [00:01<00:57, 70.81it/s]





























epoch 0 loss 18542681.2421875:   2%|█▏                                              | 101/4184 [00:01<00:57, 70.67it/s]





























epoch 0 loss 18684058.1328125:   2%|█▏                                              | 101/4184 [00:01<00:5

epoch 0 loss 22156477.48828125:   3%|█▌                                             | 141/4184 [00:02<00:57, 70.84it/s]





























epoch 0 loss 22206695.9609375:   3%|█▌                                              | 141/4184 [00:02<00:57, 70.84it/s]





























epoch 0 loss 22243718.76953125:   3%|█▌                                             | 141/4184 [00:02<00:57, 70.84it/s]





























epoch 0 loss 22293692.9609375:   3%|█▌                                              | 141/4184 [00:02<00:57, 70.84it/s]





























epoch 0 loss 22293692.9609375:   4%|█▋                                              | 149/4184 [00:02<00:57, 70.70it/s]





























epoch 0 loss 22337024.43359375:   4%|█▋                                             | 149/4184 [00:02<00:57, 70.70it/s]





























epoch 0 loss 22374610.66796875:   4%|█▋                                             | 149/4184 [00:02<00:5

epoch 0 loss 23492080.26171875:   5%|██                                             | 189/4184 [00:02<00:55, 72.53it/s]





























epoch 0 loss 23514353.826171875:   5%|██                                            | 189/4184 [00:02<00:55, 72.53it/s]





























epoch 0 loss 23531825.9375:   5%|██▎                                                | 189/4184 [00:02<00:55, 72.53it/s]





























epoch 0 loss 23531825.9375:   5%|██▍                                                | 197/4184 [00:02<00:55, 71.87it/s]





























epoch 0 loss 23551322.25:   5%|██▍                                                  | 197/4184 [00:02<00:55, 71.87it/s]





























epoch 0 loss 23570126.861328125:   5%|██▏                                           | 197/4184 [00:02<00:55, 71.87it/s]





























epoch 0 loss 23583858.94921875:   5%|██▏                                            | 197/4184 [00:02<00:5

...

epoch 1 loss 1115740.6228027344:   4%|█▋                                            | 149/4184 [00:02<00:59, 68.07it/s]





























epoch 1 loss 1124451.4694824219:   4%|█▋                                            | 149/4184 [00:02<00:59, 68.07it/s]





























epoch 1 loss 1131321.5490722656:   4%|█▋                                            | 149/4184 [00:02<00:59, 68.07it/s]





























epoch 1 loss 1139322.9313964844:   4%|█▋                                            | 149/4184 [00:02<00:59, 68.07it/s]





























epoch 1 loss 1154629.3063964844:   4%|█▋                                            | 149/4184 [00:02<00:59, 68.07it/s]





























epoch 1 loss 1154629.3063964844:   4%|█▋                                            | 157/4184 [00:02<00:58, 69.28it/s]





























epoch 1 loss 1159454.1579589844:   4%|█▋                                            | 157/4184 [00:02<00:5

epoch 1 loss 1500083.8735351562:   5%|██▏                                           | 200/4184 [00:02<00:58, 68.11it/s]





























epoch 1 loss 1504580.0551757812:   5%|██▏                                           | 200/4184 [00:02<00:58, 68.11it/s]





























epoch 1 loss 1511459.2412109375:   5%|██▏                                           | 200/4184 [00:02<00:58, 68.11it/s]





























epoch 1 loss 1516823.6743164062:   5%|██▏                                           | 200/4184 [00:02<00:58, 68.11it/s]





























epoch 1 loss 1522856.146484375:   5%|██▏                                            | 200/4184 [00:03<00:58, 68.11it/s]





























epoch 1 loss 1532820.673828125:   5%|██▏                                            | 200/4184 [00:03<00:58, 68.11it/s]





























epoch 1 loss 1540010.5678710938:   5%|██▏                                           | 200/4184 [00:03<00:5

epoch 1 loss 1854492.2717285156:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1863293.5295410156:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1867698.0939941406:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1875307.9113769531:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1883278.9797363281:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1888999.0075683594:   6%|██▋                                           | 249/4184 [00:03<00:59, 66.48it/s]





























epoch 1 loss 1892594.8200683594:   6%|██▋                                           | 249/4184 [00:03<00:5

epoch 1 loss 2207461.3967285156:   7%|███▏                                          | 291/4184 [00:04<00:59, 65.70it/s]





























epoch 1 loss 2207461.3967285156:   7%|███▎                                          | 298/4184 [00:04<00:59, 65.67it/s]





























epoch 1 loss 2214476.7014160156:   7%|███▎                                          | 298/4184 [00:04<00:59, 65.67it/s]





























epoch 1 loss 2221269.673095703:   7%|███▎                                           | 298/4184 [00:04<00:59, 65.67it/s]





























epoch 1 loss 2229700.0627441406:   7%|███▎                                          | 298/4184 [00:04<00:59, 65.67it/s]





























epoch 1 loss 2233916.6442871094:   7%|███▎                                          | 298/4184 [00:04<00:59, 65.67it/s]





























epoch 1 loss 2242479.3083496094:   7%|███▎                                          | 298/4184 [00:04<00:5

KeyboardInterrupt: 